<a href="https://colab.research.google.com/github/harjeet88/LLM_experiemnts/blob/main/RAG/RAG_on_website_with_better_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import sqlite3
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, pipeline
from bs4 import BeautifulSoup
import requests

In [48]:
# Define the model and tokenizer
model_name = "sentence-transformers/all-mpnet-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [49]:
# Setting up SQLite connection
conn = sqlite3.connect('rag_website_example.db')
cur = conn.cursor()

In [50]:
# Create table for storing documents and their embeddings
cur.execute("""
CREATE TABLE IF NOT EXISTS documents (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    content TEXT,
    embedding BLOB
);
""")
conn.commit()

In [51]:
# Function to compute embeddings
def compute_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()  # Use mean pooling to get a fixed-size embedding

In [52]:
# Function to scrape text content from a website
def scrape_website(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')
    return "\n".join([para.get_text() for para in paragraphs])

# Function to insert document into the database
def insert_document(content):
    embedding = compute_embedding(content).tobytes()  # Convert numpy array to bytes
    cur.execute("INSERT INTO documents (content, embedding) VALUES (?, ?)", (content, embedding))
    conn.commit()

In [53]:
# Function to retrieve documents based on query
def retrieve_documents(query, top_k=3):
    query_embedding = compute_embedding(query)
    cur.execute("SELECT content, embedding FROM documents")
    rows = cur.fetchall()

    # Compute cosine similarity between query embedding and document embeddings
    similarities = []
    for content, embedding in rows:
        doc_embedding = np.frombuffer(embedding, dtype=np.float32)
        similarity = np.dot(query_embedding, doc_embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_embedding))
        similarities.append((content, similarity))

    # Sort by similarity and return top_k results
    similarities.sort(key=lambda x: x[1], reverse=True)
    return [content for content, _ in similarities[:top_k]]

In [62]:
# Function to generate a response
def generate_response(query):
    retrieved_docs = retrieve_documents(query)
    combined_text = " ".join(retrieved_docs)
    # Truncate the combined text to fit within gpt2's context window
    max_length = 1024
    combined_text = combined_text[:max_length]
    generator = pipeline("text-generation", model="gpt2")
    response = generator(f"Query: {query}\nDocuments: {combined_text}\nAnswer:", max_new_tokens=200)
    return response[0]['generated_text']

In [63]:
# Example website URL
url = "https://en.wikipedia.org/wiki/Harry_Potter"
content = scrape_website(url)
insert_document(content)

# Example query
query = "Tell me about the content on the website."
response = generate_response(query)
print(response)

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: Tell me about the content on the website.
Documents: 

Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. The novels chronicle the lives of a young wizard, Harry Potter, and his friends, Hermione Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry. The main story arc concerns Harry's conflict with Lord Voldemort, a dark wizard who intends to become immortal, overthrow the wizard governing body known as the Ministry of Magic, and subjugate all wizards and Muggles (non-magical people).

The series was originally published in English by Bloomsbury in the United Kingdom and Scholastic Press in the United States.  A series of many genres, including fantasy, drama, coming-of-age fiction, and the British school story (which includes elements of mystery, thriller, adventure, horror, and romance), the world of Harry Potter explores numerous themes and includes many cultural meanings and references.[1] Majo

In [ ]:

# Close the database connection
cur.close()
conn.close()